# Setting up the environment

During the workshop, we will use LlamaIndex to build a RAG system, with Qdrant acting as the vector store. We can skip the indexing process, and simply start with a pre-built index, imported from a snapshot. However, before we start doing the actual work, we will review the underlying data and get familiar with it.

## Prerequisites

As usual, building up RAG requires a few components:

- **Qdrant instance** - obviously, no RAG without Qdrant in the loop, either local or cloud version
- **LLM** - we are going to work with OpenAI models, as they are the default of the LlamaIndex, so [please obtain an API key](https://help.openai.com/en/articles/4936850-where-do-i-find-my-openai-api-key)
- **Embedding model** - there are plenty of models out there, but an open source [`BAAI/bge-large-en`](https://huggingface.co/BAAI/bge-large-en) is today's favorite

## Setting up Qdrant

It doesn't matter if you prefer using a local Qdrant server running in a container, or our 1GB free tier cluster. Please make sure you have a running instance on hand.

### Local Docker container

If you have Docker installed on your machine, you should be able to launch an instance pretty quickly by running the following command.

In [ ]:
!docker run -d -p "6333:6333" -p "6334:6334" qdrant/qdrant

### Qdrant Cloud free tier

Another option is to [sign up for Qdrant Cloud](https://cloud.qdrant.io/login) and use the free tier 1GB cluster, which is available for everyone.

## Saving configuration

The last thing we need to set up before the start is to store all the secrets as environmental variables in the `.env` file. There is an `.env.example` we can use as a reference.

Once the configuration is done, let's try out the connection to our Qdrant instance.

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from qdrant_client import QdrantClient

import os

client = QdrantClient(
    os.environ.get("QDRANT_URL"), 
    api_key=os.environ.get("QDRANT_API_KEY"),
)
client.get_collections()

## Data import

RAG obviously needs data to work with. There are various challenges to overcome while indexing the documents, such as the chunking strategy. Another thing is creating the embeddings, which is usually a bottleneck of each system. Since this workshop is not about indexing, we're going to load the Qdrant collection from the snapshots I prepared beforehand.

In [ ]:
client.recover_snapshot(
    collection_name="hacker-news",
    # please do not modify the URL below
    location="https://snapshots.qdrant.io/workshop-rag-optimization/hacker-news-8895643013517159-2024-02-20-21-56-46.snapshot",
    wait=False, # loading a snapshot may take some time, so let's avoid a timeout
)

In [ ]:
import time

while True:
    collections = client.get_collections()
    if len(collections.collections) >= 1:
        break
    time.sleep(1.0)

collections

For now, we're going to use a single collection, but once we get to the hybrid search, there'll be another one required. Let's already import it, so we don't need to worry again.

In [ ]:
client.recover_snapshot(
    collection_name="hacker-news-hybrid",
    location="https://snapshots.qdrant.io/workshop-rag-optimization/hacker-news-hybrid-8895643013517159-2024-02-20-21-56-54.snapshot",
    wait=False,
)

In [ ]:
while True:
    collections = client.get_collections()
    if len(collections.collections) >= 2:
        break
    time.sleep(1.0)

collections